In [0]:
!pip install -q gwpy

     |████████████████████████████████| 1.4MB 2.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 2.3MB 71kB/s 


In [0]:
#Required PIPs
%%capture
!pip install --upgrade google-api-python-client
!pip install flask-ngrok
!pip install pyspellchecker
!gsutil cp gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt glove.6B.200d.txt

In [0]:
!gsutil cp gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt glove.6B.200d.txt

Copying gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt...
| [1 files][661.3 MiB/661.3 MiB]                                                
Operation completed over 1 objects/661.3 MiB.                                    


In [0]:
import string
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
%%capture
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import pickle
from keras.models import load_model

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config

from keras import models,layers

from spellchecker import SpellChecker

nltk.download('stopwords')
nltk.download('punkt')

In [0]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

In [0]:
%%capture
glove_input_file = 'glove.6B.200d.txt'
word2vec_output_file = 'glove.6B.200d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
# load the Stanford GloVe model
filename = 'glove.6B.200d.txt.word2vec'
embedding_model = KeyedVectors.load_word2vec_format(filename, binary=False)
print('Embedding Model Trained')

#Input using Image OCR using Vision API

In [0]:
import getpass
APIKEY = getpass.getpass()
#APIKEY = '<YOUR_VISION_API_KEY'

··········


In [0]:
from googleapiclient.discovery import build

In [0]:
#OCR using Vision API
def scan_text(IMAGE_URL):
  vservice = build('vision', 'v1', developerKey=APIKEY)
  request = vservice.images().annotate(body={
          'requests': [{
                  'image': {
                      'source': {
                          'imageUri': IMAGE_URL
                      }
                  },
                  'features': [{
                      'type': 'TEXT_DETECTION',
                      'maxResults': 5,
                  }]
             }],
         })
  responses = request.execute(num_retries=5)
  ocr_text = responses['responses'][0]['textAnnotations'][0]['description']
  #print(ocr_text)
  return ocr_text

#Pre-processing Data for the Model

In [0]:
#functions to pre-process data
def clean_data(text):

    essay_data = []
    # split into words
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    str1 = ""
    for i in words:
      str1 = str1 + i + " "
    return str1

def FeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def FullFeatureVector(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((1,num_features),dtype="float32")
    for essay in essays:
      essayFeatureVecs[counter] = FeatureVec(essay, model, 200)
      counter = counter + 1
    return essayFeatureVecs

def transform(data):
  essays = []
  for d in data:
    essays.append(d.split())
  return essays

def prepare_data(data):
    #clean_data = transform(data)
    DataVecs = FullFeatureVector(data, embedding_model, 200 )    
    DataVecs = np.array(DataVecs)
    DataVecs = np.reshape(DataVecs, (DataVecs.shape[0], 1, DataVecs.shape[1]))
    return (DataVecs)

In [0]:
from IPython.display import Image
Image('morphing.jpeg')


In [0]:
#Animation
#url = 'https://firebasestorage.googleapis.com/v0/b/mlkit-92eb9.appspot.com/o/Images%2FWhatsApp%20Image%202020-02-19%20at%2011.44.51%20AM%20(1).jpeg?alt=media&token=f3f5c642-ba50-4ee7-9690-1a3fbcd2dec8'
url = 'https://firebasestorage.googleapis.com/v0/b/mlkit-92eb9.appspot.com/o/Images%2FWhatsApp%20Image%202020-02-19%20at%2011.44.51%20AM.jpeg?alt=media&token=c0e14168-3822-420f-8ba3-38db6abe6878'

In [0]:
ocr_text = scan_text(url) #read the text from image
print(ocr_text)

In [0]:
ocr_text = "Flutter provides http package to consume HTTP resources. http is a Future-based library and uses await and async features. It provides many high level methods and simplifies the development of REST based mobile applications."

#Running the Flask Server

In [0]:
# app
app = Flask(__name__)

# we need to redefine our metric function in order 
# to use it when loading the model 

global graph
graph = tf.get_default_graph()
model = load_model('Model_New_.h5')
run_with_ngrok(app)   #starts ngrok when the app is run

# routes
@app.route("/",methods=['GET','POST'])
def predict():
    # get data
    data = request.get_json(force=True)
    text = clean_data(data['title'])
    spell = SpellChecker()
    words=text.split(' ')
    # find those words that may be misspelled
    misspelled = spell.unknown(words)
    misspelled = list(misspelled)
    #print(misspelled)
    data_ = []
    data_.append(text)
    data_form1 = transform(data_)
    data_vec = prepare_data(data_form1)

    data_vec_ = data_vec
    print(data)
    def your_handler():
        global graph
        with graph.as_default():
            result = model.predict(data_vec_)
        return result


    # predictions
    results = your_handler()
    # send back to browser
    output = {'results': float(results[0])}
    output['wwords']=misspelled
    return jsonify(results=output)
    

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7035ca45.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


{'title': 'hello this is Avijit Chakraborty, you know who. the secretary of sxcsa. here I am to teach you machine learning.'}


...

{'title': 'I am Avijit Chakraborty studying for the contest Nature the best place to stay in touch with you and I will be a good time to look at the end of the day of the day of the day of the day of the day of the same stage weeping willow creek road to be a good time to look at the ceiling my sis I am a very good morning I.'}


127.0.0.1 - - [25/Mar/2020 14:18:35] "POST / HTTP/1.1" 200 -


{'title': 'I am Avijit Chakraborty studying for the contest Nature the best place to stay in touch with me on my bday and and the Rains the best place for a long time ago and I will be a good day I have already filled with 4 students and the Rains I have my first visit be be able.'}


127.0.0.1 - - [25/Mar/2020 14:20:09] "POST / HTTP/1.1" 200 -


{'title': 'hello I am advanced analytical geometry vector analysis also available at UN and sons Private Limited reported in publishing books on mathematics in 1932 by the same authors'}
